# Load Data
Load up the word vector and raw FDA approved drug names.

In [1]:
import numpy as np
from gensim.models import KeyedVectors
# load the word embedding model
wv = KeyedVectors.load_word2vec_format('bio_embedding_intrinsic', binary=True)
# and our approved drug names
drugs = None
with open('fda_approved/fda_approved.processed.names') as infile:
    drugs = [l.strip() for l in infile]

# Convert Raw FDA Approved Drug Names to Vectors
Here we take the FDA approved drug names and convert them to word vectors.
First, we break them into tokens.
Then, for each drug, we get word vectors for each individual token and average them for the drug.
We drop individual tokens that are not in the vocab, modifying the drug name along with the average.
Obviously, drug names with no tokens in the vocab are dropped.

In [2]:
# we're first going to convert every multi-token drug into a word vector average
def wv_avg_tokens(tokens, wv):
    # collect the unit vectors for each token
    new_toks = list()
    unit_vecs = list()
    for t in tokens:
        # skip tokens that aren't in the embedding
        if t in wv:
            # keep the token
            new_toks.append(t)
            # scale to a unit vector
            uvec = wv[t]
            uvec = uvec / np.linalg.norm(uvec)
            unit_vecs.append(uvec)
    # now add them up if we got at least one
    if len(unit_vecs) < 1:
        return None,None
    # sum
    ret_vec = unit_vecs[0]
    for uvec in unit_vecs[1:]:
        ret_vec = ret_vec + uvec
    # and rescale
    ret_vec = ret_vec / np.linalg.norm(ret_vec)
    return new_toks,ret_vec

In [3]:
# how many raw drugs are we starting with again?
print('Starting drug count: {0}'.format(len(drugs)))
# gather up our drug vectors with the corresponding "new" names (based on token dropping)
drug_vec_tups = list()
# and track the drugs that were dropped completely
dropped_drugs = list()
for d in drugs:
    # split and average word vectors
    toks = d.split()
    new_toks,drug_vec = wv_avg_tokens(toks, wv)
    # check for complete loss of a drug, or track the new name and vector
    if drug_vec is None:
        dropped_drugs.append(d)
    else:
        new_name = ' '.join(new_toks)
        drug_vec_tups.append((new_name, drug_vec))
# and how many raw drug names were we able to convert into something
print('Converted drug count: {0}'.format(len(drug_vec_tups)))
# because we have modified drug names, we may have ended up with new dupes
tmp = list()
drug_seen = set()
for dvt in drug_vec_tups:
    if dvt[0] not in drug_seen:
        tmp.append(dvt)
        drug_seen.add(dvt[0])
drug_vec_tups = tmp
# and how many distinct converted drug names do we have
print('Final distinct drug vector count: {0}'.format(len(drug_vec_tups)))

Starting drug count: 8561
Converted drug count: 6506
Final distinct drug vector count: 5850


# Build Treatment Analogy Vectors
Now we generate treatment analogy vectors.
We use three analogies: Metformin-Diabetes, Benazepril-Hypertension, and Albuterol-Asthma.
These are the vectors we will use to rank the drugs.

In [4]:
def get_sars_treatment_analogy(seed_drug, seed_disease, wv):
    drug_v = wv[seed_drug] / np.linalg.norm(wv[seed_drug])
    dis_v = wv[seed_disease] / np.linalg.norm(wv[seed_disease])
    sars_v = wv['sars'] / np.linalg.norm(wv['sars'])
    treat_sars_vec = drug_v - dis_v + sars_v
    treat_sars_vec = treat_sars_vec / np.linalg.norm(treat_sars_vec)
    return treat_sars_vec

In [5]:
metf_diab_sars_v = get_sars_treatment_analogy('metformin', 'diabetes', wv)
benz_hypr_sars_v = get_sars_treatment_analogy('benazepril', 'hypertension', wv)
albu_asth_sars_v = get_sars_treatment_analogy('albuterol', 'asthma', wv)

# Check Top 20 Hits for Treatment Vectors
We want to manually evaluate the top 20 hits for each treatment analogy to check for drugs and drug targets.
Finding drugs and drug targets in the top hits would suggest that the treatment vectors are in the right neighborhood.

In [6]:
metf_diab_sars_top20 = wv.most_similar(positive=[metf_diab_sars_v], topn=20)
benz_hypr_sars_top20 = wv.most_similar(positive=[benz_hypr_sars_v], topn=20)
albu_asth_sars_top20 = wv.most_similar(positive=[albu_asth_sars_v], topn=20)

### Metformin/Diabetes Top 20

In [7]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in metf_diab_sars_top20]))

('sars', 0.7350299954414368)
('sars-cov', 0.5995138883590698)
('sars-3cl', 0.5938767194747925)
('sars-3clpro', 0.5917655229568481)
('sars-like', 0.588849663734436)
('sars-covs', 0.5769191980361938)
('sars-cov-induced', 0.5742612481117249)
('sars-cov-mediated', 0.5720081329345703)
('sars-cov-like', 0.5706111192703247)
('anti-sars-cov', 0.5702002048492432)
('pcsars-cov', 0.5684103965759277)
('hsars-cov', 0.5669524073600769)
('sars-co', 0.5651364326477051)
('anticoronaviral', 0.5618479251861572)
('cantharimide', 0.560847818851471)
('sar405', 0.5591368675231934)
('peramivir', 0.5569697022438049)
('norcantharidin-induced', 0.5555315017700195)
('cantharidin-mediated', 0.5551379919052124)
('delaviridine', 0.5549776554107666)


In [8]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in metf_diab_sars_top20]))

sars \\
sars-cov \\
sars-3cl \\
sars-3clpro \\
sars-like \\
sars-covs \\
sars-cov-induced \\
sars-cov-mediated \\
sars-cov-like \\
anti-sars-cov \\
pcsars-cov \\
hsars-cov \\
sars-co \\
anticoronaviral \\
cantharimide \\
sar405 \\
peramivir \\
norcantharidin-induced \\
cantharidin-mediated \\
delaviridine \\


### Benazepril/Hypertension Top 20

In [9]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in benz_hypr_sars_top20]))

('sars', 0.6842663288116455)
('sars-3cl', 0.6048033237457275)
('sars-3clpro', 0.5865695476531982)
('sars-', 0.5783016681671143)
('sars-cov', 0.5710662007331848)
('sars-covs', 0.5611740350723267)
('p-sars', 0.5571819543838501)
('sars-like', 0.5532997250556946)
('sarsp', 0.550118625164032)
('sars-cov-like', 0.5482956171035767)
('sars-hcov', 0.5416980981826782)
('anti-sars-cov', 0.5390284061431885)
('sars-s', 0.5341991186141968)
('coronavirion', 0.5340977907180786)
('lycodine', 0.5312058925628662)
('sarspp', 0.5307182669639587)
('sarse', 0.5294245481491089)
('sars-cov-s', 0.5278017520904541)
('sars-cov-', 0.5276678204536438)
('pcsars-cov', 0.5257419943809509)


In [10]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in benz_hypr_sars_top20]))

sars \\
sars-3cl \\
sars-3clpro \\
sars- \\
sars-cov \\
sars-covs \\
p-sars \\
sars-like \\
sarsp \\
sars-cov-like \\
sars-hcov \\
anti-sars-cov \\
sars-s \\
coronavirion \\
lycodine \\
sarspp \\
sarse \\
sars-cov-s \\
sars-cov- \\
pcsars-cov \\


### Albuterol/Asthma Top 20

In [11]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in albu_asth_sars_top20]))

('sars', 0.7238022089004517)
('sars-cov', 0.5882976055145264)
('csars', 0.5856403112411499)
('sars-covs', 0.5827744603157043)
('sarspp', 0.5768841505050659)
('sars-like', 0.5768048763275146)
('sars-cov-like', 0.5722691416740417)
('peramivir', 0.5660814046859741)
('vero-pipecuronium', 0.5657573938369751)
('sarsp', 0.5637783408164978)
('pancuronium-metocurine', 0.5598682761192322)
('sars-hcov', 0.5589520931243896)
('sarse', 0.5587899684906006)
('pcsars-cov', 0.5573617815971375)
('sars-3cl', 0.5541791319847107)
('p-sars', 0.5507079362869263)
('sars-3clpro', 0.548815131187439)
('sars-', 0.5467952489852905)
('sars-coronavirus', 0.5443071722984314)
('pralidoxime', 0.5440006852149963)


In [12]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in albu_asth_sars_top20]))

sars \\
sars-cov \\
csars \\
sars-covs \\
sarspp \\
sars-like \\
sars-cov-like \\
peramivir \\
vero-pipecuronium \\
sarsp \\
pancuronium-metocurine \\
sars-hcov \\
sarse \\
pcsars-cov \\
sars-3cl \\
p-sars \\
sars-3clpro \\
sars- \\
sars-coronavirus \\
pralidoxime \\


# Get Top 50 FDA Approved Drugs for Treatment Vectors
We now sort all of the FDA approved drug vectors by their cosine similarity to the analogy vectors.

In [13]:
def get_ranked_drugs_for_treat_vec(treat_vector, drug_vec_tups, wv):
    # first get the similarities for all drugs
    dvs = [dt[1] for dt in drug_vec_tups]
    drug_treat_sims = wv.cosine_similarities(treat_vector, dvs)
    # then zip them up with the drug names
    drug_sim_tups = [(dvt[0],drug_treat_sims[i]) for i,dvt in enumerate(drug_vec_tups)]
    # and sort, descending
    return sorted(drug_sim_tups, key=lambda x: x[1], reverse=True)

In [16]:
metf_diab_drugs_top50 = get_ranked_drugs_for_treat_vec(metf_diab_sars_v, drug_vec_tups, wv)[:50]
benz_hypr_drugs_top50 = get_ranked_drugs_for_treat_vec(benz_hypr_sars_v, drug_vec_tups, wv)[:50]
albu_asth_drugs_top50 = get_ranked_drugs_for_treat_vec(albu_asth_sars_v, drug_vec_tups, wv)[:50]

### Metformin/Diabetes Top 50

In [19]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in metf_diab_drugs_top50]))

('gilteritinib fumarate', 0.5596476)
('peramivir', 0.55696976)
('zanamivir', 0.5469999)
('erdafitinib', 0.528765)
('atovaquone and proguanil hydrochloride', 0.5266413)
('rimantadine hydrochloride', 0.52499855)
('delavirdine mesylate', 0.5242566)
('atazanavir sulfate and ritonavir', 0.5215566)
('cobimetinib fumarate', 0.520225)
('niclosamide', 0.51958627)
('lopinavir and ritonavir', 0.51907957)
('temsirolimus', 0.51461643)
('rilpivirine hydrochloride', 0.51084423)
('alectinib hydrochloride', 0.5094976)
('lefamulin acetate', 0.5077356)
('perphenazine and amitriptyline hydrochloride', 0.5071963)
('alogliptin and metformin hydrochloride', 0.506837)
('tamiflu', 0.50654095)
('selinexor', 0.5053856)
('amprenavir', 0.50434387)
('ibuprofen and diphenhydramine citrate', 0.5035539)
('olanzapine and fluoxetine hydrochloride', 0.50291455)
('probenecid and colchicine', 0.50227225)
('erlotinib hydrochloride', 0.5016403)
('bicalutamide', 0.5016144)
('alomide', 0.50147206)
('amantadine hydrochloride', 

In [22]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in metf_diab_drugs_top50]))

gilteritinib fumarate \\
peramivir \\
zanamivir \\
erdafitinib \\
atovaquone and proguanil hydrochloride \\
rimantadine hydrochloride \\
delavirdine mesylate \\
atazanavir sulfate and ritonavir \\
cobimetinib fumarate \\
niclosamide \\
lopinavir and ritonavir \\
temsirolimus \\
rilpivirine hydrochloride \\
alectinib hydrochloride \\
lefamulin acetate \\
perphenazine and amitriptyline hydrochloride \\
alogliptin and metformin hydrochloride \\
tamiflu \\
selinexor \\
amprenavir \\
ibuprofen and diphenhydramine citrate \\
olanzapine and fluoxetine hydrochloride \\
probenecid and colchicine \\
erlotinib hydrochloride \\
bicalutamide \\
alomide \\
amantadine hydrochloride \\
azelastine hydrochloride and fluticasone propionate \\
revefenacin \\
imipramine pamoate \\
doravirine \\
rosiglitazone maleate and metformin hydrochloride \\
nefazodone hydrochloride \\
mefloquine hydrochloride \\
abacavir sulfate and lamivudine \\
carisoprodol compound \\
triprolidine and pseudoephedrine hydrochloride

### Benazepril/Hypertension Top 50

In [20]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in benz_hypr_drugs_top50]))

('peramivir', 0.5103268)
('tamiflu', 0.4824001)
('zanamivir', 0.4645741)
('gilteritinib fumarate', 0.46427312)
('rimantadine hydrochloride', 0.46177062)
('benazepril hydrochloride', 0.45213768)
('doravirine', 0.4503398)
('galantamine hydrobromide', 0.44721016)
('cetirizine hydrochloride hives', 0.44475797)
('lanadelumab', 0.44420737)
('aliskiren hemifumarate', 0.44316927)
('desloratadine', 0.44266012)
('entacapone', 0.4421384)
('invirase', 0.44047087)
('daclatasvir dihydrochloride', 0.44043508)
('indacaterol maleate', 0.44023758)
('loratadine', 0.43836826)
('peganone', 0.43726075)
('nitazoxanide', 0.43708813)
('denavir', 0.43477327)
('triprolidine and pseudoephedrine hydrochlorides codeine', 0.43261552)
('rivastigmine', 0.43223038)
('telavancin hydrochloride', 0.43162748)
('donepezil hydrochloride', 0.43145007)
('triprolidine and pseudoephedrine hydrochlorides', 0.43077922)
('tazemetostat hydrobromide', 0.43050787)
('relenza', 0.43031728)
('benazepril hydrochloride and hydrochlorothiaz

In [23]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in benz_hypr_drugs_top50]))

peramivir \\
tamiflu \\
zanamivir \\
gilteritinib fumarate \\
rimantadine hydrochloride \\
benazepril hydrochloride \\
doravirine \\
galantamine hydrobromide \\
cetirizine hydrochloride hives \\
lanadelumab \\
aliskiren hemifumarate \\
desloratadine \\
entacapone \\
invirase \\
daclatasvir dihydrochloride \\
indacaterol maleate \\
loratadine \\
peganone \\
nitazoxanide \\
denavir \\
triprolidine and pseudoephedrine hydrochlorides codeine \\
rivastigmine \\
telavancin hydrochloride \\
donepezil hydrochloride \\
triprolidine and pseudoephedrine hydrochlorides \\
tazemetostat hydrobromide \\
relenza \\
benazepril hydrochloride and hydrochlorothiazide \\
nulojix \\
ecallantide \\
alectinib hydrochloride \\
virazole \\
levocetirizine hydrochloride \\
memantine hydrochloride and donepezil hydrochloride \\
donepezil hydrochloride and memantine hydrochloride \\
amantadine hydrochloride \\
cetirizine hydrochloride \\
comtan \\
fluvoxamine maleate \\
amlodipine besylate and benazepril hydrochlor

### Albuterol/Asthma Top 50

In [21]:
# just have a quick look at the hits and similarities
print('\n'.join([str(h) for h in albu_asth_drugs_top50]))

('peramivir', 0.56608146)
('albuterol', 0.5404904)
('albuterol sulfate', 0.5352649)
('albuterol sulfate and ipratropium bromide', 0.5332952)
('zanamivir', 0.5239428)
('rimantadine hydrochloride', 0.5186332)
('pralidoxime chloride', 0.5143588)
('meperidine and atropine sulfate', 0.5116007)
('amantadine hydrochloride', 0.505163)
('doxacurium chloride', 0.5038774)
('biperiden lactate', 0.5019073)
('atropine sulfate syringe', 0.49847904)
('gallamine triethiodide', 0.4982666)
('atropine and demerol', 0.49669114)
('colistin sulfate', 0.49553096)
('oseltamivir phosphate', 0.49364558)
('revefenacin', 0.49357465)
('dextromethorphan hydrobromide and quinidine sulfate', 0.49159467)
('conivaptan hydrochloride', 0.48869854)
('glycopyrronium tosylate', 0.4879885)
('cefiderocol sulfate tosylate', 0.48789614)
('fentanyl citrate and droperidol', 0.48749727)
('pancuronium bromide', 0.4874306)
('relenza', 0.486462)
('telavancin hydrochloride', 0.48618492)
('guaifenesin and dextromethorphan hydrobromide',

In [24]:
# and clean them up for a LaTeX table
print('\n'.join(['{0} \\\\'.format(h[0]) for h in albu_asth_drugs_top50]))

peramivir \\
albuterol \\
albuterol sulfate \\
albuterol sulfate and ipratropium bromide \\
zanamivir \\
rimantadine hydrochloride \\
pralidoxime chloride \\
meperidine and atropine sulfate \\
amantadine hydrochloride \\
doxacurium chloride \\
biperiden lactate \\
atropine sulfate syringe \\
gallamine triethiodide \\
atropine and demerol \\
colistin sulfate \\
oseltamivir phosphate \\
revefenacin \\
dextromethorphan hydrobromide and quinidine sulfate \\
conivaptan hydrochloride \\
glycopyrronium tosylate \\
cefiderocol sulfate tosylate \\
fentanyl citrate and droperidol \\
pancuronium bromide \\
relenza \\
telavancin hydrochloride \\
guaifenesin and dextromethorphan hydrobromide \\
diphenoxylate hydrochloride and atropine sulfate \\
esketamine hydrochloride \\
galantamine hydrobromide \\
naloxone hydrochloride and pentazocine hydrochloride \\
glycopyrrolate \\
levalbuterol hydrochloride \\
calfactant \\
rilpivirine hydrochloride \\
pipecuronium bromide \\
tamiflu \\
biperiden hydrochlo

# Inspect Commonalities
We want to inspect commonalities amongst the different groups to both validate consistency of the method and to find higher confidence hits.

In [30]:
metf_diab_drug_set = {h[0] for h in metf_diab_drugs_top50}
benz_hypr_drug_set = {h[0] for h in benz_hypr_drugs_top50}
albu_asth_drug_set = {h[0] for h in albu_asth_drugs_top50}

In [31]:
# look at the exact intersect of all
full_intersect = metf_diab_drug_set & benz_hypr_drug_set & albu_asth_drug_set
print('Common to all: {0}'.format(len(full_intersect)))
print('\n'.join(sorted(full_intersect)))

Common to all: 7
amantadine hydrochloride
peramivir
revefenacin
rilpivirine hydrochloride
rimantadine hydrochloride
tamiflu
zanamivir


In [32]:
# and look at the 2-set common drugs
metf_benz = metf_diab_drug_set & benz_hypr_drug_set
benz_albu = benz_hypr_drug_set & albu_asth_drug_set
albu_metf = albu_asth_drug_set & metf_diab_drug_set
two_agree_intersect = (metf_benz | benz_albu | albu_metf) - full_intersect
print('Common to two: {0}'.format(len(two_agree_intersect)))
print('\n'.join(sorted(two_agree_intersect)))

Common to two: 11
alectinib hydrochloride
donepezil hydrochloride and memantine hydrochloride
doravirine
galantamine hydrobromide
gilteritinib fumarate
memantine hydrochloride and donepezil hydrochloride
relenza
telavancin hydrochloride
triprolidine and pseudoephedrine hydrochlorides
triprolidine and pseudoephedrine hydrochlorides codeine
virazole
